In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext sql 
%pylab inline
%matplotlib inline

import pandas as pd
import os, random

import pickle,time

tmstmp = time.strftime("%m-%d-%Y",time.localtime())


Populating the interactive namespace from numpy and matplotlib


# Initialisation

## Directory paths
* TOP = path to the toplevel directory 
* TOP+/lib = path to the python source files that are required
* FIG_DIR = path to where figures are stored

In [2]:
# Set up the local source files
TOP = os.getcwd().replace('notebooks','')
LIB = TOP+'lib'
if not LIB in sys.path: sys.path.append(LIB)

DAT_DIR = TOP + '/data/'
RES_DIR = TOP + '/results/'
FIG_DIR = TOP + '/figs/'


## Load packages
Open organtox_v1 MongoDB - if this fails then check your mongodb installation

In [3]:
from organtox import *

DB = openMongo(host='pb.epa.gov',user='ishah',passwd='ishah',db='organtox_v1')
GNR = openMongo(host='pb.epa.gov',user='ishah',passwd='ishah',db='genra_v3')
DB.collection_names()

/usr/local/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[u'chm_fp',
 u'ml_lr_v1',
 u'ml_study_lr_v1',
 u'bio_fp',
 u'ml_summary_v2',
 u'ml_study_summary_v1',
 u'ml_summary_v1',
 u'ml_run_v1',
 u'tox_fp',
 u'ml_study_run_v1',
 u'tox2_fp']

# Toxicity data

Extend the toxicity fp to include only study level hits

In [9]:
GNR.tox_fp.find_one()

{u'_id': ObjectId('56d9da9814fd09250ac2fe7f'),
 u'casrn': u'103-90-2',
 u'dsstox_cid': u'DTXCID606',
 u'name': u'Acetaminophen',
 u'tox1': {u'ds': [u'SUB:liver',
   u'SUB:lymph_node',
   u'MGR:offspring_survival-early',
   u'SUB:uterus',
   u'SUB:body_weight',
   u'CHR:thyroid_gland',
   u'MGR:pituitary_gland',
   u'SUB:mortality',
   u'MGR:liver',
   u'MGR:reproductive_performance',
   u'SUB:testes',
   u'MGR:body_weight',
   u'SAC:body_weight',
   u'SUB:ovary',
   u'CHR:tissue_nos',
   u'CHR:kidney',
   u'SUB:thymus',
   u'MGR:brain',
   u'CHR:body_weight',
   u'MGR:sperm_morphology',
   u'SAC:food_consumption',
   u'SUB:kidney',
   u'SUB:food_consumption',
   u'SUB:clinical_signs',
   u'CHR:parathyroid_gland'],
  u'n': 25},
 u'toxq1': {u'CHR:[not_in_list]': 0.0,
  u'CHR:abdominal_cavity': 0.0,
  u'CHR:adrenal_gland': 0.0,
  u'CHR:artery_(general)': 0.0,
  u'CHR:auditory_startle_reflex_habituation': 0.0,
  u'CHR:bile_duct': 0.0,
  u'CHR:blood': 0.0,
  u'CHR:blood_vessel': 0.0,
  u'CH

In [10]:
DB.tox_fp.find_one()

{u'_id': ObjectId('58591fa8072e60736e265a1b'),
 u'dsstox_cid': u'DTXCID10281',
 u'tox_fpn1': {u'ds': [u'SUB:Lung',
   u'SUB:Mammary Gland',
   u'SUB:Intestine Large',
   u'MGR:Mammary Gland',
   u'SUB:Uterus',
   u'SUB:Thyroid Gland',
   u'SUB:Clinical Signs',
   u'SUB:Lymph Node',
   u'MGR:Thyroid Gland',
   u'SUB:Eye',
   u'SUB:Seminal Vesicle',
   u'MGR:Uterus',
   u'SUB:Spinal cord',
   u'MGR:Kidney',
   u'SUB:Intestine Small',
   u'MGR:Seminal Vesicle',
   u'MGR:Ovary',
   u'MGR:Testes',
   u'MGR:Brain',
   u'SUB:Vagina',
   u'SUB:Urinary Bladder',
   u'MGR:Heart',
   u'SUB:Thymus',
   u'SUB:Esophagus',
   u'SUB:Pancreas',
   u'SUB:Brain',
   u'SUB:Penis',
   u'SUB:Adrenal Gland',
   u'SUB:Skeletal Muscle',
   u'MGR:Epididymis',
   u'SUB:Heart',
   u'MGR:Lung',
   u'MGR:Stomach',
   u'SUB:Pituitary Gland'],
  u'n': 34},
 u'tox_fpp1': {u'ds': [u'SUB:Testes',
   u'SUB:Kidney',
   u'SUB:Liver',
   u'SUB:Bone Marrow',
   u'SUB:Spleen',
   u'SUB:Stomach'],
  u'n': 6}}

## Build study level descriptors
1. Create collection tox2_fp
2. Define tox_fpp2: study is positive if tox_fp.tox_fpp1 has STUDY
3. Define tox_fpn2: study is negative if tox_fp.tox_fpn1 has STUDY and not tox_fpp2


In [28]:
from util.jupyter import *

DB.tox2_fp.drop()

for FP in log_progress(DB.tox_fp.find({},dict(_id=0)),size=DB.tox_fp.count()):
    FPN1 = FP.pop('tox_fpn1')
    FPP1 = FP.pop('tox_fpp1')
    
    Pos = set([i.split(':')[0] for i in FPP1['ds']])
    Neg = set([i.split(':')[0] for i in FPN1['ds']])
    Neg2= list(Neg.difference(Pos))
    
    FPP2= dict(ds=list(Pos),n=len(Pos))
    FPN2= dict(ds=list(Neg2),n=len(Neg2))
    
    FP['tox_fpp2']=FPP2
    FP['tox_fpn2']=FPN2
    
    # Dose
    X = GNR.tox_fp.find_one(dict(dsstox_cid=FP['dsstox_cid']),dict(_id=0,toxq1=1))
    if X:
        Toxq = pd.DataFrame(X['toxq1'].items(),columns=['effect','dose'])
        Toxq['study']=Toxq.effect.str[:3]
        Toxq['dose']=Toxq.dose.astype(np.float)
        
        Dose=Toxq.query("dose>0").groupby('study').aggregate(dict(dose=dict(mn=np.min,md=np.median,mx=np.max)))
        C = [i+'_'+j for i,j in list(Dose.columns)]
        Dose.columns=C
        FP['toxq2']=Dose.reset_index().to_dict('records')
        
    DB.tox2_fp.insert_one(FP)

In [29]:
DB.tox2_fp.find_one({'tox_fpp2.n':{'$gte':3}})

{u'_id': ObjectId('59a5965b072e601d90cd5f4c'),
 u'dsstox_cid': u'DTXCID10194',
 u'tox_fpn2': {u'ds': [], u'n': 0},
 u'tox_fpp2': {u'ds': [u'MGR', u'CHR', u'SUB', u'DEV'], u'n': 4},
 u'toxq2': [{u'dose_md': 167.5,
   u'dose_mn': 167.5,
   u'dose_mx': 1100.0,
   u'study': u'CHR'},
  {u'dose_md': 205.5, u'dose_mn': 76.0, u'dose_mx': 1003.0, u'study': u'DEV'},
  {u'dose_md': 4500.0,
   u'dose_mn': 336.0,
   u'dose_mx': 4500.0,
   u'study': u'MGR'},
  {u'dose_md': 1025.4,
   u'dose_mn': 25.0,
   u'dose_mx': 4466.7,
   u'study': u'SUB'}]}

## See if this builds correct datasets

In [48]:
from organtox import *

DS1 = getToxDataSet('DEV',MDB=DB,col_tox='tox2_fp',tox_pos_ds='tox_fpp2.ds',tox_neg_ds='tox_fpn2.ds')

In [49]:
[[k,v.shape] for k,v in DS1.iteritems()]

[['bio', (429, 780)],
 ['tox', (581, 1)],
 ['bc', (424, 2782)],
 ['bct', (424, 1151)],
 ['chm', (476, 2002)],
 ['ct', (424, 371)]]

In [95]:
Tox={}
for X in DB.tox2_fp.find():
    fp = tuple(X['tox_fpp2']['ds'])
    if Tox.has_key(fp):
        Tox[fp]+=1
    else:
        Tox[fp] =1
        

In [96]:
Tox

{(): 63,
 (u'CHR',): 51,
 (u'CHR', u'DEV'): 33,
 (u'CHR', u'NEU', u'SUB', u'DEV'): 2,
 (u'CHR', u'REP', u'DEV'): 4,
 (u'CHR', u'REP', u'DEV', u'SUB'): 1,
 (u'CHR', u'SUB'): 65,
 (u'CHR', u'SUB', u'DEV'): 66,
 (u'CHR', u'SUB', u'DEV', u'REP'): 2,
 (u'DEV',): 64,
 (u'DNT',): 20,
 (u'DNT', u'CHR'): 3,
 (u'DNT', u'CHR', u'DEV'): 5,
 (u'DNT', u'CHR', u'DEV', u'MGR'): 7,
 (u'DNT', u'CHR', u'SUB'): 4,
 (u'DNT', u'CHR', u'SUB', u'DEV'): 6,
 (u'DNT', u'CHR', u'SUB', u'DEV', u'MGR'): 8,
 (u'DNT', u'DEV'): 3,
 (u'DNT', u'SUB', u'DEV'): 1,
 (u'DNT', u'SUB', u'DEV', u'MGR'): 1,
 (u'MGR',): 30,
 (u'MGR', u'CHR'): 9,
 (u'MGR', u'CHR', u'DEV'): 26,
 (u'MGR', u'CHR', u'DEV', u'DNT'): 8,
 (u'MGR', u'CHR', u'NEU', u'SUB'): 1,
 (u'MGR', u'CHR', u'NEU', u'SUB', u'DEV'): 3,
 (u'MGR', u'CHR', u'REP'): 1,
 (u'MGR', u'CHR', u'SUB'): 20,
 (u'MGR', u'CHR', u'SUB', u'DEV'): 163,
 (u'MGR', u'CHR', u'SUB', u'DEV', u'DNT'): 22,
 (u'MGR', u'CHR', u'SUB', u'DEV', u'REP'): 3,
 (u'MGR', u'DEV'): 15,
 (u'MGR', u'DEV', u'

# Run ML Analysis

## Parallelisation

In [6]:
from organtox import *

initParallel(parallel_machine='pb_parallel')

d_view

<DirectView [0, 1, 2, 3,...]>

In [7]:
resp=d_view.push(dict(TOP=TOP,LIB=LIB))

d_view.execute("""
import sys
if not LIB in sys.path: sys.path.append(LIB)
from organtox import *
DB = openMongo(host='pb.epa.gov',user='ishah',passwd='ishah',db='organtox_v1')
""")


<AsyncResult: execute:finished>

In [7]:
#d_view.map(lambda x: DB.ml_study_run_v1.count(), range(10))

In [10]:
from organtox import *
import random
P_ALL = []

DB.ml_study_run_v1.drop()
n_np = 0
Outcomes = DB.tox2_fp.distinct('tox_fpp2.ds')
for tox in Outcomes:
    RN0=[]
    # Get one seed for each toxicity classification
    seed = random.randint(1,1e6)      
    
    # Get the data 
    DS0 = getToxDataSet(tox,MDB=DB,col_tox='tox2_fp',tox_pos_ds='tox_fpp2.ds',tox_neg_ds='tox_fpn2.ds')

    N_p,N_n = (DS0['tox'][tox]==1).sum(),(DS0['tox'][tox]==0).sum()
    
    if N_p==0 or N_n==0: continue
    print "> Tox ",tox,N_p,'+', N_n,'-',time.strftime("%H:%M",time.localtime())
    sys.stdout.flush()
    sys.stderr.flush()
    
    for n_p in range(N_n,N_p,20):
        for dt in ['bio','chm','ct','bc','bct']:
            RN0.append(dict(tox_class=tox,descriptor_type=dt,
                            n_neg=N_n,n_pos=n_p,rand_seed=seed))
            
    DB.ml_study_run_v1.insert_many(RN0)


> Tox  SUB 496 + 31 - 19:26
> Tox  CHR 520 + 19 - 19:26
> Tox  DEV 526 + 55 - 19:26
> Tox  DNT 92 + 19 - 19:26
> Tox  MGR 338 + 101 - 19:27


In [11]:
print DB.ml_study_run_v1.count()
#pd.DataFrame(list(DB.ml_study_run_v1.find(dict(tox_class='DEV',descriptor_type='bio'),dict(_id=0))))

450


In [12]:
DB.ml_study_lr_v1.drop()

WORK = [(w['tox_class'],w['descriptor_type'],w['n_pos'],w['n_neg'],w['rand_seed']) 
        for w in DB.ml_study_run_v1.find()]
shuffle(WORK)

print "Starting ML Analysis to assign toxicity " + time.strftime("%d/%m/%Y %H:%M",time.localtime())
print "Data sets: ", DB.ml_study_run_v1.count()


P = lb_view.map(lambda (tox_i,dt_i,np_i,nn_i,rs_i): 
                    runStudyToxML(tox_i,dt_i,np_i,nn_i,rs_i,
                                  col_tox='tox2_fp',tox_pos_ds='tox_fpp2.ds',tox_neg_ds='tox_fpn2.ds',
                                  ss_iters=10,cv_iters=2,cv_nfolds=5,
                                  n_ds_min=2,n_ds_max=22,n_ds_step=2,
                                  Col_ds=None,Col_lr=DB.ml_study_lr_v1,MDB=DB),
                    WORK)

print "Ending ML Analysis to assign activity " + time.strftime("%d/%m/%Y %H:%M",time.localtime())
#send_email(txt="Done",subj="ToxBrain Done")

Starting ML Analysis to assign toxicity 29/08/2017 19:28
Data sets:  450


CompositeError: one or more exceptions from call to method: <lambda>
[7:apply]: DataError: No numeric types to aggregate
[15:apply]: DataError: No numeric types to aggregate
[28:apply]: DataError: No numeric types to aggregate